<hr><hr><center>Objective:&nbsp;<strong><em>Fix K8s Pod in ImagePullBackOff State</em></strong></center>
<p>1)&nbsp;<a href="#1" target="_self" rel="noopener">Get list of pods in ImagePullBackOff State</a><br>2)&nbsp;<a href="#2" target="_self" rel="noopener">Extract Events of the pods</a><br>3)&nbsp;<a href="#3" target="_self" rel="noopener">Check registry accessibility</a></p>
<p>An <code>ImagePullBackOff</code> error occurs when a Pod startup fails to pull the specified image. The reasons could be Non-Existent of the repository or Permission to Access the repository issues. This runbook helps to walk through the steps involved in debugging such a Pod.&nbsp;We'll then create the steps required to resolve the issue - learning how to use unSkript at the same time.</p>
<hr>

<h3 id="Get-List-of-Pods-in-ImagePullBackOff-State&para;"><a id="1" target="_self" rel="nofollow"></a>Get List of Pods in ImagePullBackOff State<a class="jp-InternalAnchorLink" href="#Get-List-of-Pods-in-CrashLoopBackOff-State" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="#Get-List-of-Pods-in-ImagePullBackOff-State&para;" target="_self">&para;</a></h3>
<p>This action fetches a list of the pods in ImagePullBackOff State. This action will consider <code>namespace</code> as&nbsp;<strong> all&nbsp;</strong>if no namespace is given.</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>namespace</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>imagepullbackoff_pods</code></p>
</blockquote>

In [4]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/aSsYlIGQhO8" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

<h3 id="Create-List-of-commands-to-get-Events&para;">Create List of commands to get Events<a class="jp-InternalAnchorLink" href="#Examine-the-Events" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="#Create-List-of-commands-to-get-Events&para;" target="_self">&para;</a></h3>
<p>Examine the output from Step 1👆,&nbsp; and create a list of commands for each pod in a namespace that is found to be in the ImagePullBackOff State</p>
<blockquote>
<p>This action captures the following ouput:&nbsp;<code>imagepullbackoff_pods</code></p>
</blockquote>

<h3 id="Gather-information-of-the-pods">Extract Events of the pods<a class="jp-InternalAnchorLink" href="#Gather-information-of-the-pods" target="_self">&para;</a></h3>
<p>This action describes events for a list of unhealthy pods obtained in Step 1.</p>
<blockquote>
<p>This action captures the following ouput: <code>describe_output</code></p>
</blockquote>

In [5]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field

from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()
    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "SandboxEKS",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "9b49b1bd-38cc-47e9-b2c8-4911b6a4bdb1"
}''')
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "kubectl_command": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "[ f\\"kubectl describe pod {x} -n {namespace} | grep -A 10 Events\\" for x in imagepullbackoff_pods ]",
    "iter_parameter": "kubectl_command"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(imagepullbackoff_pods)!=0",
    "condition_result": true
    }''')
task.configure(outputName="describe_output")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

<h3 id="Examine-the-Events&para;">Examine the Events<a class="jp-InternalAnchorLink" href="#Examine-the-Events" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="#Examine-the-Events&para;" target="_self">&para;</a></h3>
<p>This Custom Action searches Known errors .&nbsp;The well known errors are listed in the error_msgs variable. If&nbsp;there is a new error message that was found, it can be added to the list.</p>

In [6]:
import re

"""
This Custom Action searches Known errors in the podEvents variable.
The well known errors are listed in the error_msgs variable. If
there is a new error message that was found, you can add it to this
list and the next run, the runbook will catch that error.
"""

def check_msg(msg, err):
    return re.search(err, msg)

error_msgs = ["repository (.*) does not exist or no pull access",
              "manifest for (.*) not found",
              "pull access denied, repository does not exist or may require authorization",
             "Back-off pulling image (.*)"]
cause_found = False
result = ''
for key, msg in describe_output.items():
    for err in error_msgs:
        result = check_msg(msg, err)
        if result is not None:
            print("PROBABLE CAUSE: ", f"{result.string}")
            cause_found = True

repoLocation = ''
if cause_found is False:
    print("ERROR MESSAGE : \n", all_describe_info)
else:
    try:
        repoLocation = result.groups()[0]
    except:
        pass
    else:
        print("Image Repo Location : ", repoLocation)

<h3 id="Create-List-of-commands-to-get-Exit-Code&para;">Check Registry Accessibility<a class="jp-InternalAnchorLink" href="#Examine-the-Events" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="#Create-List-of-commands-to-get-Exit-Code&para;" target="_self">&para;</a></h3>
<p>From the output from Step 2B👆check if the repoLocation is accessible.</p>

In [18]:
from IPython.display import Markdown as md

if repoLocation is not None:
    display(md(f"**Please verify Repo {repoLocation} is accessible from the K8S POD**"))

In [19]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/qXS3ILkti0s" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

<p>Step 3a: Add this to an Action (add -&gt; Action)</p>
<p><strong id="docs-internal-guid-a21dbc63-7fff-a731-b8e0-45e1efa43d7f">patchCommand= "kubectl patch pod image-pullback -n " + namespace + ' -p \'{"spec":{"containers":[{"name":"image-pullback-container", "image":"debian"}]}}\''</strong></p>
<p>&nbsp;</p>
<p>Step 3b: Search actions on the Riught menu for "Kubectl Command." Drag this action in, add your K8s credentials.</p>
<p>&nbsp;</p>
<p>Add this to the Kubectl Command</p>
<p><strong>patchCommand</strong></p>
<p>&nbsp;</p>
<p>Step 3c:&nbsp;</p>
<p>Drag in a second "Kubectl Command" action, add your K8s credentials.</p>
<p>Add this to the Kubectl Command:</p>
<p><strong id="docs-internal-guid-8bbd08ae-7fff-d143-ec83-5fc85433d193">f'kubectl get pods -n {namespace}'</strong></p>
<p>&nbsp;</p>